# Partie 2 : Analyse de Données Tabulaires avec le Machine Learning Traditionnel

## 2.1. Contexte et Objectifs
Dans ce module, votre objectif sera de démontrer que pour les données structurées et tabulaires, les algorithmes de ML "classiques" sont souvent plus performants et plus simples à mettre en oeuvre que le Deep Learning. Nous allons pivoter vers un cas d'usage de détection de fraude sur les transactions financières, qui introduit le défi crucial des classes fortement déséquilibrées.

À la fin de ce module, votre objectif sera de :
- Mener une analyse exploratoire de données (EDA) avec Pandas et Seaborn.
- Identifier et gérer un problème de déséquilibre de classes.
- Entraîner et évaluer un `RandomForestClassifier` de Scikit-learn.
- Maîtriser l'interprétation des métriques d'évaluation pertinentes : précision, rappel (recall) et F1-score.

## 2.2. Le Jeu de Données : Détection de Fraude
Pourriez-vous utiliser le dataset Credit Card Fraud Detection disponible sur Kaggle? Il contient des transactions anonymisées (`V1` à `V28`) et une variable cible `Class` (1 pour fraude, 0 sinon). Le défi principal est le déséquilibre extrême : moins de 0.2% des transactions sont frauduleuses .

## 2.3. Étapes d’implémentation

### 2.3.1 : Chargement et Analyse Exploratoire (EDA)
Commencez par charger les données et explorer leurs caractéristiques (via les Dataframes de pandas, fonction head, info, describe ; via seaborn.countplot). Vous m’afficherez ces différentes valeurs et métriques.

Capture d’écran n°5 : Les informations demandées précédemment

### 2.3.2 : Préparation des Données
Division Entraînement/Test :
Maintenant, préparez les données pour le modèle.
- Mise à l'échelle : La colonne `Amount` a une échelle très différente des autres. Il est essentiel de la mettre à l'échelle.
= Division Entraînement/Test : Séparez les données en veillant à conserver la même proportion de classes dans les deux ensembles (`stratify=y`).

### 2.3.3 : Entraînement du Modèle
Votre tâche est maintenant d'entraîner un `RandomForestClassifier`. Pour gérer le déséquilibre, pourriez-vous utiliser le paramètre `class_weight='balanced'` ? Il pénalisera davantage les erreurs sur la classe minoritaire (fraude).

Capture d’écran n°5 : Phase d’entraînement en console ou autre

### 2.3.4 : Évaluation Rigoureuse
Enfin, évaluez le modèle en utilisant les métriques appropriées : matrice de confusion, heatmap (via seaborn), classification_report (via scikit-learn/sklearn)

## P2 Questions pour le rapport :
1. Expliquez le rôle du paramètre `class_weight='balanced'`. Quelles autres techniques auriez-vous pu utiliser pour gérer le déséquilibre des classes ?
2. Quel type d’apprentissage et quel type de problème est-ce ? Argumentez.
3. Donnez un exemple de régression lié à ce type de données, mettez un exemple via argumentation et image.